# Classes

Overall syntax:

```python
class SomeClass(Base1, Base2, ...):
    """Useful documentation."""
    class_attr = ...
    
    def __init__(self, some_arg):
        self.instance_attr = some_arg
        
    def some_method(self):
        pass
```

`self` is an explicit reference to a class instance.

Example:

In [2]:
class Counter:
    """I count. That is all."""
    def __init__(self, initial=0):  # constructor
        self.value = initial        # object attribute
        
    def increment(self):
        self.value += 1
        
    def get(self):
        return self.value

In [3]:
c = Counter(42)

In [4]:
c.increment()

In [5]:
c.get()

43

## "Private" Methods and Attributes

There are no such things in Python. Everything is public. Developers just follow naming conventions to inform about internal class members.

In [12]:
class Noop:
    some_attribute = 42
    _interal_attribute = []  # name of the attribute starts with _

Some programmers use `__` (double underscore) instead of `_` to really emphasize the purpose.

In [18]:
class Noop:
    __internal_attribute = []

In [19]:
Noop.__internal_attribute

AttributeError: type object 'Noop' has no attribute '__internal_attribute'

But a class member is still publicly accessible. I don't think it worth it.

In [20]:
Noop._Noop__internal_attribute

[]

## Class Attributes

⚠️  Only for educational purposes. This class is really broken.

In [26]:
from collections import deque


class MemorizingDict(dict):
    _history = deque(maxlen=10)  # class attribute
    
    def set(self, key, value):
        self._history.append(key)
        self[key] = value
        
    def get_history(self):
        return self._history

In [27]:
d = MemorizingDict({"foo": 42})
d.set("baz", 100500)
d.get_history()

deque(['baz'])

In [28]:
d = MemorizingDict()
d.set("boo", 500100)
d.get_history()

deque(['baz', 'boo'])

## Magic Methods

In [29]:
class Noop:
    """I do nothing at all."""

In [30]:
Noop.__doc__

'I do nothing at all.'

In [31]:
Noop.__name__

'Noop'

In [35]:
Noop.__base__

object

In [32]:
Noop.__module__

'__main__'

In [33]:
noop = Noop()
noop.__class__

__main__.Noop

In [34]:
noop.__dict__  # all attributes of an object

{}

In [36]:
# Class is also an object
Noop.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'I do nothing at all.',
              '__dict__': <attribute '__dict__' of 'Noop' objects>,
              '__weakref__': <attribute '__weakref__' of 'Noop' objects>})

In [37]:
Noop.__class__

type

ℹ️ _We can work with object or class attributes in the same way we work with a dictionary._

In [39]:
noop.some_attribute = 42
noop.__dict__

{'some_attribute': 42}

In [40]:
# just for example
noop.__dict__["some_attribute"] = 100500
noop.some_attribute

100500

In [41]:
del noop.some_attribute

In [42]:
noop.__dict__

{}

🔮 Some dark magic:

In [45]:
noop.number = 42
noop.string = "Hello"
noop.__dict__

{'number': 42, 'string': 'Hello'}

In [46]:
del noop.__dict__
noop.__dict__

{}

Attribute lookup happens dynamically at runtime.

In [44]:
vars(noop)  # noop.__dict__

{}

## Bind Methods

In [47]:
class SomeClass:
    def do_something(self):
        print("Doing something.")

In [49]:
SomeClass().do_something  # method is bind to the class instance

<bound method SomeClass.do_something of <__main__.SomeClass object at 0x10ad89110>>

In [50]:
SomeClass().do_something()

Doing something.


In [51]:
SomeClass.do_something  # unbind method is just a function

<function __main__.SomeClass.do_something(self)>

In [52]:
instance = SomeClass()

In [53]:
SomeClass.do_something(instance)

Doing something.


## Properties

Properties are useful when we want to **evaluate** them **during a call**.

In [58]:
from os.path import dirname


class Path:
    def __init__(self, current):
        self.current = current
        
    def __repr__(self):
        return f"Path({self.current})"
    
    @property
    def parent(self):
        return Path(dirname(self.current))  # evaluate at every call

In [59]:
p = Path("/Users/akrisanov/.zshrc")

In [60]:
p.parent

Path(/Users/akrisanov)

### Getters, Setters, Deleters

In [63]:
class BigDataModel:
    def __init__(self):
        self._params = []
        
    @property
    def params(self):
        return self._params
    
    @params.setter
    def params(self, new_params):
        assert all(map(lambda p: p > 0, new_params))
        self._params = new_params
        
    @params.deleter
    def params(self):
        del self._params

In [64]:
model = BigDataModel()
model.params = [0.1, 0.5, 0.4]
model.params

[0.1, 0.5, 0.4]

## Inheritance

In [65]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial

In [66]:
class OtherCounter(Counter):
    def get(self):
        return self.value

In [67]:
c = OtherCounter()  # Counter.__init__
c.get()             # OtherCounter.get()

0

In [68]:
c.value            # c.__dict__["value"]

0

### super()

[Documentation](https://docs.python.org/3/library/functions.html#super)

In [69]:
class Counter:
    all_counters = []
    
    def __init__(self, initial=0):
        self.__class__.all_counters.append(self)
        self.value = initial

In [70]:
class OtherCounter(Counter):
    def __init__(self, initial=0):
        self.initial = initial
        super().__init__(initial)

In [71]:
oc = OtherCounter()
vars(oc)

{'initial': 0, 'value': 0}

### Predicates

In [75]:
class A:
    pass


class B(A):
    pass

In [76]:
isinstance(B(), A)

True

In [77]:
class C:
    pass

In [78]:
isinstance(B(), (A, C))

True

In [79]:
isinstance(B(), A) or isinstance(B(), C)

True

In [80]:
# Why not just write type(B()) == A?
type(B())

__main__.B

### Multiple Inheritance

[C3 linearization](https://en.wikipedia.org/wiki/C3_linearization)

[The C3 algorithm, under control of a total order](http://doc.sagemath.org/html/en/reference/misc/sage/misc/c3_controlled.html#)

In [81]:
class A:
    def f(self):
        print("A.f")
        
        
class B:
    def f(self):
        print("B.f")
        
        
class C(A, B):
    pass

In [82]:
C().f()

A.f


In [83]:
C.mro()  # list(C.__mro__)

[__main__.C, __main__.A, __main__.B, object]

### Mixins

[Wikipedia](https://en.wikipedia.org/wiki/Mixin)

## Class Decorators

[PEP 3129](https://www.python.org/dev/peps/pep-3129/)

Class decorators work in the same way as function decorators, but they act on classes rather than functions.

```python
@deco
class Noop:
    pass


# vs

class Noop:
    pass


Noop = deco(Noop)
```

In this case, a decorator is a function that takes a class as an argument and returns another (modified) class.

We can use class decorators instead of using mixins.

### Singleton

In [86]:
import functools


def singleton(cls):
    instance = None
    
    @functools.wraps(cls)
    def inner(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance
    
    return inner

ℹ️ _`wraps` also copies `__dict__` from the decorated class._

In [87]:
@singleton
class Noop:
    """I do nothing at all"""

In [88]:
id(Noop())

4477455184

In [89]:
id(Noop())

4477455184

### Deprecation Warning

In [90]:
import warnings

In [97]:
def deprecated(cls):
    orig_init = cls.__init__
    
    @functools.wraps(cls.__init__)
    def new_init(self, *args, **kwargs):
        warnings.warn(
            cls.__name__ + " is deprecated.",
            category=DeprecationWarning)
        orig_init(self, *args, **kwargs)
        
    cls.__init__ = new_init
    return cls

In [98]:
@deprecated
class Counter:
    def __init__(self, initial=0):
        self.value = initial

In [99]:
c = Counter()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Counter is deprecated.
  


## Magic Methods

[Special method names](https://docs.python.org/3/reference/datamodel.html#special-method-names)

### `__getattr__`

In [100]:
class Noop:
    pass

In [101]:
Noop().foobar

AttributeError: 'Noop' object has no attribute 'foobar'

In [102]:
class Noop:
    def __getattr__(self, name):
        return name # identity

In [103]:
Noop().foobar

'foobar'

### `__setattr__` and `__delattr__`

Unlike `__getattr__` these methods are called for all attributes, even existing ones.

In [104]:
# Just for educational purposes!

class Guarded:
    guarded = []
    
    def __setattr__(self, name, value):
        assert name not in self.guarded
        super().__setattr__(name, value)

In [105]:
class Noop(Guarded):
    guarded = ["foobar"]
    
    def __init__(self):
        self.__dict__["foobar"] = 42  # skip __setattr__ method call

Also check out:

- [Bunch library](https://github.com/dsc/bunch)
- [Packt: Bunch](https://subscription.packtpub.com/book/application_development/9781788830829/1/ch01lvl1sec18/bunch)

In [107]:
class Noop:
    some_attribute = 42

In [108]:
noop = Noop()
getattr(noop, "some_attribute")

42

In [112]:
getattr(noop, "some_other_attribute", 100500)

100500

In [113]:
setattr(noop, "some_other_attribute", 100500)

In [114]:
delattr(noop, "some_other_attribute")

### Comparison Methods

```python
instance.__eq__(other)  # instance == other
instance.__ne__(other)  # instance != other
instance.__lt__(other)  # instance < other
instance.__le__(other)  # instance <= other
instance.__gt__(other)  # instance > other
instance.__ge__(other)  # instance >= other
```

In [115]:
import functools


@functools.total_ordering  # <- helps a lot, but it's not that fast
class Counter:
    def __eq__(self, other):  # required
        return self.value == other.value
    
    def __lt__(self, other):  # or <=, >, >=
        return self.value < other.value

### `__call__`

In [116]:
class Identity:
    def __call__(self, x):
        return x

In [117]:
Identity()(42)

42

Usecase?

**Class-based decorator with arguments:**

Sometimes it can be useful and help readability. The initialization and the decorator logic are separated.

💡 When you have a long function-base decorator, think about class-based decorator.

In [118]:
class trace:
    def __init__(self, handle):
        self.handle = handle
        
    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs, file=self.handle)
            return func(*args, **kwargs)
        return inner

In [120]:
import sys


@trace(sys.stderr)
def identity(x):
    return x

In [121]:
identity(42)

identity (42,) {}


42

### Conversion to String

In [122]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __repr__(self):
        return f"Counter({self.value})"
    
    def __str__(self):
        return f"Counter to {self.value}"

c = Counter(42)

In [124]:
c  # __repr__

Counter(42)

In [125]:
print(c)  # __str__

Counter to 42


### __format__

In [129]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __format__(self, format_spec):
        return self.value.__format__(format_spec)

In [130]:
c = Counter(42)

In [131]:
"Counted to {:b}".format(c)

'Counted to 101010'

### __hash__

```python
x is y <=> hash(x) == hash(y)
```

Couple advice:

- Implement `__hash__` method together with `__eq__`
    - `__hash__` must satisfy `x == y => hash(x) == hash(y)`
- For mutable objects, it's ok to define only `__eq__` method

### __bool__

Helps to check for truthy / falsy.

In [133]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial
        
    def __bool__(self):
        return bool(self.value)

In [134]:
c = Counter()

In [135]:
if not c:
    print("No counts yet.")

No counts yet.
